In [65]:
import collections

from Bio import pairwise2
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

from tf.app import use
A = use('etcbc/bhsa', hoist=globals())
Fmt, Tmt, Lmt = F, T, L

B = use('dt-ucph/sp', version='3.4', hoist=globals())
Fsp, Tsp, Lsp = F, T, L

C = use('etcbc/dss', version='1.9', hoist=globals())
Fdss, Tdss, Ldss = F, T, L

del F, T, L

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
scroll,1001,1428.81,100
lex,10450,129.14,94
fragment,11182,127.91,100
line,52895,27.04,100
clause,125,12.85,0
cluster,101099,6.68,47
phrase,315,5.10,0
word,500995,2.81,99
sign,1430241,1.00,100


In [67]:
PENTATEUCH_BOOKS = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy']
ALL_BOOK_NAMES = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]

In [68]:
QSP_SCROLLS = {'1Qisaa', '1QisaaI', '1QisaaII', '2Q3', '4Q13', '4Q20', '2Q7', '4Q27', '1Q4', '2Q12', '4Q37', '4Q38', '4Q38a', '4Q40', '4Q53',
               '4Q57', '2Q13', '4Q78', '4Q80', '4Q82', '4Q128', '4Q129', '4Q134', '4Q135', '4Q136',
                '4Q137', '4Q138', '4Q139', '4Q140', '4Q141', '4Q142', '4Q143', '4Q144', '4Q158', '4Q364',
                '4Q365', '4Q96', '4Q111', '4Q109', '11Q5', '11Q6', '11Q7', '11Q8'}

In [69]:
class Book:
    def __init__(self, manuscript, book_name, F, T, L):
        self.manuscript = manuscript
        self.book_name = book_name
        self.F = F
        self.T = T
        self.L = L
        if self.manuscript in {'MT', 'SP'}:
            self.verse_g_cons, self.word2char = self.prepare_book_data()
        else:
            self.verse_g_cons, self.word2char = self.prepare_dss_book_data()
        self.verse_text_dict = self.make_verse_text()
        
    def prepare_book_data(self):
        verse_g_cons = collections.defaultdict(list)
        word2char = collections.defaultdict(list)
    
        for book_node in eval(self.F + ".otype.s('book')"):
            book_name = eval(self.T + '.sectionFromNode(book_node)[0]')
            if book_name != self.book_name:
                continue
            words = eval(self.L + ".d(book_node, 'word')")
            for w in words:
                bo, ch, ve = eval(self.T + '.sectionFromNode(w)')
                g_cons = eval(self.F + '.g_cons.v(w)')
                trailer = eval(self.F + '.trailer.v(w)')
                if trailer:
                    trailer = ' '
                verse_g_cons[(bo, ch, ve)].append(g_cons + trailer)
                for char in g_cons:
                    word2char[(bo, ch, ve)].append(w)
        return verse_g_cons, word2char
    
    def prepare_dss_book_data(self):
        verse_g_cons = collections.defaultdict(list)
        word2char = collections.defaultdict(list)
        
        for scr in eval(self.F + ".otype.s('scroll')"):
            if eval(self.T + '.scrollName(scr)') == self.manuscript:
                words = eval(self.L + ".d(scr, 'word')")
                for w in words:
                    bo = eval(self.F + '.book_etcbc.v(w)')
                    if bo != self.book_name:
                        continue            
                    ch = eval(self.F + '.chapter.v(w)')
                    ve = eval(self.F + '.verse.v(w)')
                
                    g_cons = eval(self.F + '.g_cons.v(w)')
                    after = eval(self.F + '.after.v(w)')

                    if after is None:
                        after = ''
                    if g_cons:
                        verse_g_cons[(bo, int(ch), int(ve))].append(g_cons + after)
                        for char in g_cons:
                            word2char[(bo, int(ch), int(ve))].append(w)
        return verse_g_cons, word2char
    
    def make_verse_text(self):
        return {section : ''.join(g_conss).strip() for (section, g_conss) in self.verse_g_cons.items()}

In [70]:
class AllBooks:
    def __init__(self):
        self.data = {}
        

In [71]:
def align_verses(str_1, str_2):
        
    seq1 = Seq(str_1)
    seq2 = Seq(str_2)
    
    alignments = pairwise2.align.globalxx(seq1, seq2)
    
    seq1_al = (alignments[0][0]).strip(' ')
    seq2_al = (alignments[0][1]).strip(' ')
        
    return seq1_al, seq2_al

In [72]:
def make_alignments(verse_text1, verse_text2):
    alignments_dict = {}

    for section, text1 in verse_text1.items():
        try:
            text2 = verse_text2[section]
            alignment1, alignment2 = align_verses(text1, text2)
            alignments_dict[section] = (alignment1, alignment2)
        except:
            continue
    return alignments_dict

In [73]:
def collect_matching_words(alignments_dict, word2char1, word2char2):
    man1_man2_dict = collections.defaultdict(list)

    for section, (al1, al2) in alignments_dict.items():
        man1_idx = 0
        man2_idx = 0
    
        word_chars1 = word2char1[section]
        word_chars2 = word2char2[section]
        for char1, char2 in zip(al1, al2):
            if char1 not in {' ', '-'}:
                man1_word = word_chars1[man1_idx]
                man1_idx += 1
            
            if char2 not in {' ', '-'}:
                man2_word = word_chars2[man2_idx]
                man2_idx += 1
            
            if char1 not in {' ', '-'} and char2 not in {' ', '-'}:
                man1_man2_dict[man1_word].append(man2_word)
                
    return man1_man2_dict

In [74]:
def most_frequent(List):
    return max(set(List), key = List.count)

# Prepare MT and SP texts

Produce dictionary mt_sp_matches which has mt words nodes as keys and matching word numbers from SP as values.

In [75]:
# prepare mt and sp books
MANUSCRIPTS = ['MT', 'SP']
all_books = AllBooks()
for book_name in ALL_BOOK_NAMES:
    book = Book('MT', book_name, 'Fmt', 'Tmt', 'Lmt')
    all_books.data[('MT', book_name)] = book
    
    if book_name in PENTATEUCH_BOOKS:
        book = Book('SP', book_name, 'Fsp', 'Tsp', 'Lsp')
        all_books.data[('SP', book_name)] = book

# Match words

In [76]:
dat = pd.read_csv('../data/ptca_qal.csv', sep='\t')
dat_dss = dat[~dat.scroll.isin(['MT', 'SP'])]
scroll_book_combinations = list(set(zip(dat_dss.scroll, dat_dss.book)))
dat_dss.shape

(1062, 35)

In [77]:
for manuscript, book_name in scroll_book_combinations:
    book = Book(manuscript, book_name, 'Fdss', 'Tdss', 'Ldss')
    all_books.data[(manuscript, book_name)] = book

In [78]:
def make_matching_word_dict(book_name, all_books):

    all_match_dicts = {}

    matching_book = book_name
    matching_manuscripts = [scr for scr, bo in all_books.data.keys() if bo == matching_book]

    for man1 in matching_manuscripts:
        for man2 in matching_manuscripts:
            if man1 != man2:
            
                man1_verse_texts = all_books.data[(man1, matching_book)].verse_text_dict
                man2_verse_texts = all_books.data[(man2, matching_book)].verse_text_dict

                alignments_dict = make_alignments(man1_verse_texts, man2_verse_texts)
    
                man1_word2char = all_books.data[(man1, matching_book)].word2char
                man2_word2char = all_books.data[(man2, matching_book)].word2char
 
                matching_words_dict = collect_matching_words(alignments_dict, man1_word2char, man2_word2char)

                matches = {}
                for man1_word, man2_list in matching_words_dict.items():
                    man2_word = most_frequent(man2_list)
                    matches[man1_word] = man2_word
                all_match_dicts[((man1, matching_book), (man2, matching_book))] = matches
                
    return all_match_dicts, matching_manuscripts

In [79]:
def read_dataset(file):

    return pd.read_csv(file, sep='\t')

In [80]:
class MaterData:
    def __init__(self, man1, man2, section, lex, mater_val1, mater_val2, tf_id1, tf_id2, g_cons1, g_cons2):
        self.man1 = man1
        self.man2 = man2
        self.section = section
        self.lex = lex
        self.mater_val1 = mater_val1
        self.mater_val2 = mater_val2
        self.tf_id1 = tf_id1
        self.tf_id2 = tf_id2
        self.g_cons1 = g_cons1
        self.g_cons2 = g_cons2

In [81]:
def collect_matching_cases(matching_manuscripts, matching_book, dat):

    all_mater_datasets = {}

    for man in matching_manuscripts:
        mater_data = dat[(dat.book == matching_book) & (dat.scroll == man)]
        all_mater_datasets[(man, matching_book)] = mater_data
        
    return all_mater_datasets

In [82]:
def collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, matching_book):

    manuscript_mater_match = collections.defaultdict(list)
    manuscripts = set()

    for idx, man in enumerate(matching_manuscripts):
        for idx2, man2 in enumerate(matching_manuscripts):
            if idx < idx2:
            
                matching_ids = all_match_dicts[((man, matching_book), (man2, matching_book))]
                man_data = all_mater_datasets[(man, matching_book)]
                man2_data = all_mater_datasets[(man2, matching_book)]
                for _, row in man_data.iterrows():
                
                    tf_id = row.tf_id
                    lex, typ, has_vl = row.lex, row.type, row.has_vowel_letter
                    g_cons1 = row.g_cons
                    section = (row.book, row.chapter, row.verse)
                    if man == 'SP':
                        tf_id = tf_id - 100000

                    matching_tf_id = matching_ids.get(tf_id, None)
                
                    if not matching_tf_id:
                        continue
                    
                    if man2 == 'SP':
                        matching_tf_id = matching_tf_id + 100000
                
                    man2_row = man2_data[(man2_data.tf_id == matching_tf_id) & (man2_data.lex == lex) & (man2_data.type == typ)]
                    
                    if not man2_row.shape[0]:
                        continue
                    has_vl2 = man2_row.has_vowel_letter.iloc[0]
                    g_cons2 = man2_row.g_cons.iloc[0]
                 
                    mater_data = MaterData(man, man2, section, lex, has_vl, has_vl2, tf_id, matching_tf_id, g_cons1, g_cons2)
                
                    manuscript_mater_match[man].append(mater_data)
                    manuscripts.add(man)
                    manuscripts.add(man2)
                    
    return manuscript_mater_match, manuscripts

In [83]:
def register_similarities_with_mt(manuscripts, mt_ids, manuscript_mater_match):

    mater_value_dict = {0: -1,
                    1: 1}

    mater_match_array = np.zeros((len(manuscripts), len(mt_ids)))

    for dat_object in manuscript_mater_match['MT']:
        print(dat_object.g_cons1, dat_object.g_cons2)
        
        other_man = dat_object.man2
        mt_tfid = dat_object.tf_id1
        other_man_tfid = dat_object.tf_id2
    
        mt_mater = dat_object.mater_val1
        other_man_mater = dat_object.mater_val2
    
        mt_idx = man2idx['MT']
        other_man_idx = man2idx[other_man]
    
        mt_mater_value = mater_value_dict[mt_mater]
        other_man_mater_value = mater_value_dict[other_man_mater]
    
        mt_tf_id = mt_tf2idx[mt_tfid]
    
        mater_match_array[mt_idx, mt_tf_id] = mt_mater_value
        mater_match_array[other_man_idx, mt_tf_id] = other_man_mater_value
        
    return mater_match_array

In [84]:
def get_parallels(manuscript_mater_match, hif_match_dict):
    for scroll in manuscript_mater_match.keys():
        for dat_object in manuscript_mater_match[scroll]:
            man1 = dat_object.man1
            man2 = dat_object.man2
            if man1 == 'MT' or man2 == 'MT':
            
                hif_match_dict[dat_object.tf_id2] = dat_object.tf_id1
                hif_match_dict[dat_object.tf_id1] = dat_object.tf_id2
        
    return hif_match_dict


In [85]:
def count_parallel_cases(mater_match_array):

    mater_arr = np.zeros((2, mater_match_array.shape[1]))

    for col_idx in range(mater_match_array.shape[1]):
        col = mater_match_array[:, col_idx]
        col_counts = collections.Counter(col)
        with_vowel_count = col_counts.get(1, 0)
        without_vowel_count = col_counts.get(-1, 0)
        mater_arr[0, col_idx] = with_vowel_count
        mater_arr[1, col_idx] = without_vowel_count
        
    return mater_arr

In [86]:
file_name = '../data/ptca_qal.csv'
all_bib_books = [Tmt.sectionFromNode(bo)[0] for bo in Fmt.otype.s('book')]
dat = read_dataset(file_name)

hif_match_dict = {}

for book in all_bib_books:
    #if book != 'Isaiah':
    #    continue
    print(book)
    all_match_dicts, matching_manuscripts = make_matching_word_dict(book, all_books)
    all_mater_datasets = collect_matching_cases(matching_manuscripts, book, dat)
    manuscript_mater_match, manuscripts = collect_mater_data(matching_manuscripts, all_match_dicts, all_mater_datasets, book)
    
    #print(manuscript_mater_match)
    #man2idx = {man:idx for idx, man in enumerate(manuscripts)}
    #mt_ids = sorted(list({data.tf_id1 for data in manuscript_mater_match['MT']}))
    #mt_tf2idx = {tf_id:idx for idx, tf_id in enumerate(mt_ids)}
    #idx2mt_tf = {v:k for k, v in mt_tf2idx.items()}
    
    hif_match_dict = get_parallels(manuscript_mater_match, hif_match_dict)
    print(len(hif_match_dict))
    

Genesis
260
Exodus
599
Leviticus
754
Numbers
1089
Deuteronomy
1554
Joshua
1572
Judges
1574
1_Samuel
1584
2_Samuel
1598
1_Kings
1600
2_Kings
1600
Isaiah
2412
Jeremiah
2458
Ezekiel
2472
Hosea
2474
Joel
2482
Amos
2502
Obadiah
2509
Jonah
2529
Micah
2569
Nahum
2589
Habakkuk
2597
Zephaniah
2611
Haggai
2611
Zechariah
2617
Malachi
2629
Psalms
2808
Job
2810
Proverbs
2824
Ruth
2828
Song_of_songs
2828
Ecclesiastes
2830
Lamentations
2844
Esther
2844
Daniel
2853
Ezra
2853
Nehemiah
2853
1_Chronicles
2853
2_Chronicles
2853


In [87]:
hif_match_dict

{505819: 395,
 395: 1938142,
 505953: 530,
 530: 505953,
 506010: 586,
 586: 506010,
 506023: 599,
 599: 506023,
 506039: 616,
 616: 506039,
 506286: 865,
 865: 506286,
 506309: 888,
 888: 506309,
 506343: 921,
 921: 506343,
 506357: 935,
 935: 1938320,
 506670: 1243,
 1243: 506670,
 506684: 1257,
 1257: 506684,
 507125: 1702,
 1702: 1938460,
 507201: 1779,
 1779: 1938541,
 507245: 1821,
 1821: 1938584,
 507255: 1831,
 1831: 507255,
 507327: 1905,
 1905: 507327,
 507335: 1913,
 1913: 507335,
 507350: 1928,
 1928: 507350,
 507434: 2011,
 2011: 507434,
 507446: 2023,
 2023: 507446,
 507457: 2034,
 2034: 507457,
 507459: 2036,
 2036: 507459,
 508609: 3193,
 3193: 508609,
 508732: 3320,
 3320: 508732,
 508867: 3453,
 3453: 508867,
 508885: 3474,
 3474: 508885,
 509291: 3893,
 3893: 509291,
 509531: 4135,
 4135: 509531,
 509603: 4209,
 4209: 509603,
 509645: 4250,
 4250: 509645,
 509772: 4375,
 4375: 509772,
 511461: 5944,
 5944: 511461,
 511508: 5991,
 5991: 511508,
 511860: 6339,
 6339: 5

In [29]:
len(hif_match_dict)

2722

In [88]:
all_scrolls = set(dat_dss.scroll) 
non_qsp_scrolls = all_scrolls.difference(QSP_SCROLLS)
non_qsp_data = dat_dss[dat_dss.scroll.isin(non_qsp_scrolls)]
non_qsp_pent = non_qsp_data[non_qsp_data.book.isin(PENTATEUCH_BOOKS)]
non_qsp_pent.shape

(152, 35)

In [89]:
for index, row in non_qsp_data.iterrows():
    tf_id = row.tf_id, 
    scroll = row.scroll, 
    has_vl = row.has_vowel_letter
    book = row.book
    print(tf_id)
    
    if tf_id[0] in hif_match_dict:
        corr_id = hif_match_dict[tf_id[0]]
        corr_row = dat[dat.tf_id == corr_id]
        print(scroll, corr_row.scroll.iloc[0], book, has_vl, corr_row.has_vowel_letter.iloc[0])
        
    else:
        print(scroll, book, row.chapter, row.verse, row.lex, row.g_cons)
        

(1892841,)
('1Q5',) MT Deuteronomy 0 0
(1892923,)
('1Q5',) Deuteronomy 29 17 PNH[ PNH
(1893141,)
('1Q5',) MT Deuteronomy 1 0
(1893286,)
('1Q5',) MT Deuteronomy 0 0
(1893492,)
('1Q5',) MT Deuteronomy 1 1
(1919227,)
('1Q8',) MT Isaiah 1 1
(1919431,)
('1Q8',) MT Isaiah 0 0
(1919753,)
('1Q8',) MT Isaiah 0 0
(1919766,)
('1Q8',) MT Isaiah 0 0
(1919784,)
('1Q8',) MT Isaiah 0 0
(1920412,)
('1Q8',) Isaiah 24 18 <LH[ <WLH
(1920738,)
('1Q8',) MT Isaiah 0 0
(1920779,)
('1Q8',) Isaiah 28 16 JSD[ JWSD
(1920957,)
('1Q8',) MT Isaiah 1 0
(1921208,)
('1Q8',) MT Isaiah 0 0
(1921642,)
('1Q8',) MT Isaiah 1 1
(1922039,)
('1Q8',) MT Isaiah 0 0
(1922043,)
('1Q8',) MT Isaiah 1 1
(1922066,)
('1Q8',) MT Isaiah 0 0
(1922163,)
('1Q8',) MT Isaiah 0 0
(1922360,)
('1Q8',) MT Isaiah 1 0
(1922765,)
('1Q8',) MT Isaiah 1 0
(1922767,)
('1Q8',) MT Isaiah 1 0
(1922772,)
('1Q8',) Isaiah 44 24 <FH[ <FH
(1922778,)
('1Q8',) MT Isaiah 1 0
(1922788,)
('1Q8',) MT Isaiah 1 0
(1922805,)
('1Q8',) MT Isaiah 1 0
(1922828,)
('1Q8',) MT 

('Mur88',) MT Jonah 0 0
(2094282,)
('Mur88',) MT Jonah 1 1
(2094630,)
('Mur88',) MT Micah 0 0
(2094780,)
('Mur88',) MT Micah 1 1
(2094817,)
('Mur88',) MT Micah 1 1
(2094848,)
('Mur88',) MT Micah 0 0
(2094851,)
('Mur88',) MT Micah 1 1
(2094874,)
('Mur88',) MT Micah 0 0
(2094877,)
('Mur88',) MT Micah 0 0
(2094917,)
('Mur88',) MT Micah 0 0
(2095006,)
('Mur88',) MT Micah 0 1
(2095020,)
('Mur88',) MT Micah 0 0
(2095057,)
('Mur88',) MT Micah 0 0
(2095102,)
('Mur88',) MT Micah 0 0
(2095214,)
('Mur88',) MT Micah 0 0
(2095577,)
('Mur88',) MT Micah 1 1
(2095898,)
('Mur88',) MT Micah 0 0
(2096111,)
('Mur88',) MT Micah 0 0
(2096174,)
('Mur88',) MT Micah 0 0
(2096205,)
('Mur88',) MT Micah 0 0
(2096353,)
('Mur88',) MT Micah 0 0
(2096472,)
('Mur88',) MT Nahum 0 0
(2096477,)
('Mur88',) MT Nahum 0 0
(2096482,)
('Mur88',) MT Nahum 1 1
(2096509,)
('Mur88',) MT Nahum 1 1
(2096546,)
('Mur88',) MT Nahum 1 0
(2096578,)
('Mur88',) MT Nahum 0 0
(2096585,)
('Mur88',) MT Nahum 0 0
(2096600,)
('Mur88',) Nahum 1 9